#  Train a GPT-2 Text-Generating Model w/ GPU For Free 

by [Max Woolf](http://minimaxir.com)

*Last updated: February 14th, 2021*

Retrain an advanced text generating neural network on any text dataset **for free on a GPU using Collaboratory** using `gpt-2-simple`!

For more about `gpt-2-simple`, you can visit [this GitHub repository](https://github.com/minimaxir/gpt-2-simple). You can also read my [blog post](https://minimaxir.com/2019/09/howto-gpt2/) for more information how to use this notebook!


To get started:

1. Copy this notebook to your Google Drive to keep it and save your changes. (File -> Save a Copy in Drive)
2. Make sure you're running the notebook in Google Chrome.
3. Run the cells below:


In [ ]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

TensorFlow 1.x selected.
     |████████████████████████████████| 489.6 MB 22 kB/s 
     |████████████████████████████████| 463 kB 46.0 MB/s 
     |████████████████████████████████| 1.3 MB 41.4 MB/s 
     |████████████████████████████████| 5.8 MB 41.5 MB/s 


## GPU

Colaboratory uses either a Nvidia T4 GPU or an Nvidia K80 GPU. The T4 is slightly faster than the old K80 for training GPT-2, and has more memory allowing you to train the larger GPT-2 models and generate more text.

You can verify which GPU is active by running the cell below.

In [ ]:
!nvidia-smi

Wed Dec  8 12:19:25 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80/P4 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [ ]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 439Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 8.36Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 622Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:09, 50.1Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 515Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 9.91Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 9.21Mit/s]


## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
gpt2.mount_gdrive()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Uploading a Text File to be Trained to Colaboratory

In the Colaboratory Notebook sidebar on the left of the screen, select *Files*. From there you can upload files:

![alt text](https://i.imgur.com/TGcZT4h.png)

Upload **any smaller text file**  (<10 MB) and update the file name in the cell below, then run the cell.

In [ ]:

#file_name = 'A00-1001.txt'
file_name = '2110.12501_original.txt'

If your text file is larger than 10MB, it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

In [ ]:
gpt2.copy_file_from_gdrive(file_name)

In [ ]:
#get the length of the dataset and the first k =50 words
file = open(file_name, encoding='latin1')
data = file.read()
words = data.split()

print('Number of words in text file :', len(words))
substring = data.split()[0:50]
#print(substring)
seed = ' '.join(substring)
print(seed)

Number of words in text file : 1623
ï»¿Abstract Relation extraction in the biomedical domain is a challenging task due to a lack of labeled data and a long-tail distribution of fact triples. Many works leverage distant supervision which automatically generates labeled data by pairing a knowledge graph with raw textual data. Distant supervision produces noisy labels and


## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [ ]:
#do not run the following two cells,if you are loading an existing checkpoint

In [ ]:
#IMPORTANT NOTE: If you want to rerun this cell, 
#restart the VM first (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=1000,
              restore_from='fresh',
              run_name='run1',
              print_every=10,
              sample_every=1000,
              save_every=500
              )

gpt2.copy_checkpoint_to_gdrive(run_name='run1')

Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt
Loading dataset...


100%|██████████| 1/1 [00:00<00:00, 3284.50it/s]

dataset has 2142 tokens
Training...


[10 | 49.47] loss=1.38 avg=1.38
[20 | 93.70] loss=0.26 avg=0.82
[30 | 137.69] loss=0.03 avg=0.55
[40 | 181.59] loss=0.02 avg=0.42
[50 | 225.50] loss=0.03 avg=0.34
[60 | 269.36] loss=0.02 avg=0.28
[70 | 313.25] loss=0.01 avg=0.24
[80 | 357.06] loss=0.02 avg=0.22
[90 | 400.91] loss=0.01 avg=0.19
[100 | 444.81] loss=0.01 avg=0.17
[110 | 488.83] loss=0.01 avg=0.16
[120 | 532.71] loss=0.01 avg=0.15
[130 | 576.60] loss=0.01 avg=0.13
[140 | 620.62] loss=0.01 avg=0.12
[150 | 664.58] loss=0.01 avg=0.12
[160 | 708.51] loss=0.01 avg=0.11
[170 | 752.76] loss=0.01 avg=0.10
[180 | 796.89] loss=0.01 avg=0.10
[190 | 840.87] loss=0.01 avg=0.09
[200 | 884.83] loss=0.01 avg=0.09
[210 | 928.80] loss=0.01 avg=0.08
[220 | 972.80] loss=0.01 avg=0.08
[230 | 1016.74] loss=0.01 avg=0.08
[240 | 1060.72] loss=0.01 avg=0.07
[250 | 1104.69] loss=0.01 avg=0.07
[260 | 1148.79] loss=0.01 avg=0.07
[270 | 1192.73] loss=0.01 avg=0.07
[280 | 1236.68] loss=0.01 avg=0.06
[290 | 1280.62] loss=0.01 avg=0.06
[300 | 1324.53] lo

After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name='run1')

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [ ]:
#if you are loading an existing checkpoint, start running from this cell
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [ ]:
# dont run this if you are not loadning an existing checkpoint
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

Loading checkpoint checkpoint/run1/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-1000


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [ ]:
gpt2.generate(sess, run_name='run1')

Abstract This paper proposes a technique for adding a new source or target language to an existing multilingual NMT model without retraining it on the initial set of languages. It consists in replacing the shared vocabulary with a small language-specific vocabulary and fine-tuning the new embeddings on the new language’s parallel data. Some additional language-specific components may be trained to improve performance (e.g., Transformer layers or adapter modules). Because the parameters of the original model are not modified, its performance on the initial languages does not degrade. We show on two sets of experiments (small-scale on TED Talks, and largescale on ParaCrawl) that this approach performs as well or better as the more costly alternatives; and that it has excellent zero-shot performance: training on English-centric data is enough to translate between the new language and any of the initial languages. Introduction Multilingual Neural Machine Translation models are trained on m

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [ ]:
gpt2.generate(sess,
              length=100,
              temperature=0.7,
              prefix=seed,
              nsamples=1,
              batch_size=1
              )

ï»¿Abstract Relation extraction in the biomedical domain is a challenging task due to a lack of labeled data and a long-tail distribution of fact triples. Many works leverage distant supervision which automatically generates labeled data by pairing a knowledge graph with raw textual data. Distant supervision produces noisy labels and requires additional techniques, such as multi-instance learning (MIL), to denoise the training signal. However, MIL requires multiple instances of data and struggles with very long-tail datasets such as those found in the biomedical domain. In this work, we propose a novel reformulation of MIL for biomedical relation extraction that abstractifies biomedical entities into their corresponding semantic types. By grouping entities by types, we are better able to take advantage of the benefits of MIL and further denoise the training signal. We


For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [ ]:
#gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())
gen_file = 'Output.txt'

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=100,
                      temperature=0.7,
                      prefix=seed,
                      nsamples=1,
                      batch_size=1
                      )

In [ ]:
# No need to run these individual code segments
#code segment to make the fake file length equal to original file
output_file_name = 'Output.txt'
output_file = open(output_file_name, encoding='latin1')
output_data = output_file.read()
output_words = output_data.split()

print('Number of words in output file :', len(output_words))
output_substring = output_words[-51:-1]
output_seed = ' '.join(output_substring)
print(output_seed)
final_substring = output_words[-len(output_words):-52]
String_final = ' '.join(final_substring)
print(String_final)

Number of words in output file : 711
considered one language just a few years ago, the most closely related languages in this group are Czech and Slovak. This fact has led us to an experiment with automatic translation between Czech and Slovak. It was clear that application of a similar method to that one used in the
Using examples of the transfer-based MT system between Czech and Russian RUSLAN and the word-for-word MT system with morphological disambiguation between Czech and Slovak CESILKO we argue that for really close languages it is possible to obtain better translation quality by means of simpler methods. The problem of translation to a group of typologically similar languages is also discussed here. 3. Translation and localization 2.1 A pivot language Localization of products and their documentation is a great problem for any company, which wants to strengthen its position on foreign language market, especially for companies producing various kinds of software. The amounts of t

In [ ]:
# No need to run these individual code segments
while(len(output_final_words) < len(words) ):

  with open('Output_final.txt', 'a') as f:
    f.write(String_final)
    f.close()

  gen_file = 'Output.txt'
  gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=1023,
                      temperature=0.7,
                      prefix=output_seed,
                      nsamples=1,
                      batch_size=1
                      )
  
  output_file_name = 'Output.txt'
  output_file = open(output_file_name, encoding='latin1')
  output_data = output_file.read()
  output_words = output_data.split()
  print('Number of words in output file :', len(output_words))
  output_substring = output_words[-51:-1]
  output_seed = ' '.join(output_substring)
  #print(output_seed)
  final_substring = output_words[-len(output_words):-52]
  String_final = ' '.join(final_substring)
  #print(String_final)
  output_final_file = open('Output_final.txt', encoding='latin1')
  output_final_data = output_final_file.read()
  output_final_words = output_final_data.split()
  print('Number of words in final output file :', len(output_final_words))
  
  #print("Do it")

In [ ]:
#Everything together in this code. Therefore no need to run the previous three
String_final=''
output_seed = seed

while True:
  with open('Output_final.txt', 'a') as f:
    f.write(String_final)
    f.close()

  gen_file = 'Output.txt'
  gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=1023,
                      temperature=0.7,
                      prefix=output_seed,
                      nsamples=1,
                      batch_size=1
                      )
  
  output_file_name = 'Output.txt'
  output_file = open(output_file_name, encoding='latin1')
  output_data = output_file.read()
  output_words = output_data.split()
  print('Number of words in output file :', len(output_words))
  output_substring = output_words[-51:-1]
  output_seed = ' '.join(output_substring)
  #print(output_seed)
  final_substring = output_words[-len(output_words):-52]
  String_final = ' '.join(final_substring)
  #print(String_final)

  output_final_file = open('Output_final.txt', encoding='latin1')
  output_final_data = output_final_file.read()
  output_final_words = output_final_data.split()
  print('Number of words in final output file :', len(output_final_words))

  if (len(output_final_words) > len(words) ):
    break

Number of words in output file : 787
Number of words in final output file : 1
Number of words in output file : 757
Number of words in final output file : 735
Number of words in output file : 755
Number of words in final output file : 1439
Number of words in output file : 760
Number of words in final output file : 2141


In [ ]:
#code to make the fake file length equal to original file length
difference = len(output_final_words) - len(words)
#print(difference)
first = len(output_final_words)
last = difference 
Final_fake_text = output_final_words[-first:-last]
Final_fake_text_for_the_fake_file = ' '.join(Final_fake_text)
print(Final_fake_text_for_the_fake_file)
with open('Final_fake_file.txt', 'w') as f:
  f.write(Final_fake_text_for_the_fake_file)
  f.close()


ï»¿ÃÂ¯ÃÂ»ÃÂ¿Abstract Relation extraction in the biomedical domain is a challenging task due to a lack of labeled data and a long-tail distribution of fact triples. Many works leverage distant supervision which automatically generates labeled data by pairing a knowledge graph with raw textual data. Distant supervision produces noisy labels and requires additional techniques, such as multi-instance learning (MIL), to denoise the training signal. However, MIL requires multiple instances of data and struggles with very long-tail datasets such as those found in the biomedical domain. In this work, we propose a novel reformulation of MIL for biomedical relation extraction that abstractifies biomedical entities into their corresponding semantic types. By grouping entities by types, we are better able to take advantage of the benefits of MIL and further denoise the training signal. We show this reformulation, which we refer to as abstractified multi-instance learning (AMIL), improves perfor

In [ ]:
# may have to run twice to get file to download
files.download(gen_file)

## Generate Text From The Pretrained Model

If you want to generate text from the pretrained model, not a finetuned model, pass `model_name` to `gpt2.load_gpt2()` and `gpt2.generate()`.

This is currently the only way to generate text from the 774M or 1558M models with this notebook.

In [ ]:
model_name = "774M"

gpt2.download_gpt2(model_name=model_name)

Fetching checkpoint: 1.05Mit [00:00, 354Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 131Mit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, 279Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 3.10Git [00:23, 131Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 380Mit/s]                                                
Fetching model.ckpt.meta: 2.10Mit [00:00, 226Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 199Mit/s]                                                       


In [ ]:
sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, model_name=model_name)

W0828 18:37:58.571830 139905369159552 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


Loading pretrained model models/774M/model.ckpt


In [ ]:
gpt2.generate(sess,
              model_name=model_name,
              prefix="The secret of life is",
              length=100,
              temperature=0.7,
              top_p=0.9,
              nsamples=5,
              batch_size=5
              )

The secret of life is that it's really easy to make it complicated," said Bill Nye, the host of the popular science show "Bill Nye the Science Guy." "And this is one of the reasons why we all need to be smarter about science, because we can't keep up with the amazing things that are going on all the time."

While Nye is correct that "everything that's going on all the time" is making the world a better place, he misses the point. This is not
The secret of life is in the rhythm of the universe. It's not a mystery. It's not a mystery to me. It's the nature of the universe. It's the beauty of the universe. It's the way the universe works. It's the way the universe is. It's the way the universe is going to work. It's the way the universe is. It's the way the universe is. It's the way the universe is. It's the way the universe is. It's the way
The secret of life is in the universe.


-

The Red Devil

It's the end of the world as we know it, and the only thing that can save us is a band of 

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [ ]:
!kill -9 -1

# LICENSE

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.